### Icefabric QuickStart

This notebook is meant to showcase the icefabric repo services, versioning, tools, and capabilities as an enterprise data services product from NGWPC

The notebook is broken up into the following sections:

